In [12]:
import datetime



def get_current_date():
    ''' Gives you system local time '''
    return str(datetime.datetime.now().astimezone())

get_current_date()




'2025-12-14 12:54:49.769122+05:30'

In [2]:
import sys
import os

# Add parent directory (project root) to path
sys.path.insert(0, os.path.abspath('..'))

In [ ]:
import json
from typing import Literal
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from langgraph.graph import END, StateGraph
from langgraph.graph.graph import START
from brain.agent_state import MessagesState
from brain.llm_config.config import LLMConfig
from brain.llm_config.prompts import get_system_prompt
from brain.tools.time_tools import get_current_date
from server.endpoints import user_messages


llm_config = LLMConfig()
tools = [
    get_current_date
]

    # 3. Create the lookup dictionary
tool_by_name = {tool.name: tool for tool in tools}
    
llm_with_tools = llm_config.llm.bind_tools(tools)

system_message = SystemMessage(content=get_system_prompt())
messages = [HumanMessage(content="I want book an appointment on next monday")]
full_message_list = [system_message] + messages

# print(full_message_list)
def llm_call(state: MessagesState):
    print("--- Entering LLM Call Node ---") # <-- Add this
    system_message = SystemMessage(content=get_system_prompt())
    
    # 2. Construct the full list of messages: [SystemMessage, *HistoryMessages]
    full_message_list = [system_message] + state["messages"]
    print("------------------------------------------------------")
    for m in full_message_list:
        m.pretty_print()
    print("------------------------------------------------------")
    # 3. Invoke the LLM with the complete message list
    response_message = llm_with_tools.invoke(full_message_list)
    print(f"LLM Response: {response_message}")
    return {
        "messages": [response_message]
    }

def tool_node(state: dict):
    '''Excecute the tool call'''
    print("--- Entering Environment (Tool) Node ---")
    result = [] + state["messages"]
    print(f"Tool Call Request: {state['messages'][-1].tool_calls}")
    for tool_call in state["messages"][-1].tool_calls:
        tool = tool_by_name[tool_call["name"]]
        observation = tool.invoke(tool_call["args"])
        print(observation)
        observation_str: str
        if isinstance(observation, (dict, list)):
            # Convert structured output to a JSON string
            observation_str = json.dumps(observation)
        elif observation is not None:
            # Convert all other non-None objects (like Pydantic models, dates) to string
            observation_str = str(observation)
        else:
            observation_str = "Tool executed successfully with no direct output." # Handle None/empty output
        result.append(
            ToolMessage(content=observation_str,tool_call_id=tool_call["id"],name=tool_call["name"])
        )
    print("--- Exiting Environment (Tool) Node ---")
    return {"messages": result}

def should_continue(state: MessagesState) -> Literal["environment", END]:
    """Decide if we should continue the loop or stop based upon whether the LLM made a tool call"""
    

    messages = state["messages"]
    last_message = messages[-1]
    print("------------------------------------------------------")
    print(last_message.tool_calls)
    print("------------------------------------------------------")
    # If the LLM makes a tool call, then perform an action
    if last_message.tool_calls:
        return "Action"
    # Otherwise, we stop (reply to the user)

    return END
response = llm_with_tools.invoke(full_message_list)
agent_builder = StateGraph(MessagesState)
agent_builder.add_node("llm_call", llm_call)
agent_builder.add_node("environment", tool_node)

agent_builder.add_edge(START, "llm_call")
agent_builder.add_conditional_edges(
    "llm_call",
    should_continue,
    {
        # Name returned by should_continue : Name of next node to visit
        "Action": "environment",
        END: END,
    }
)
agent_builder.add_edge("environment", "llm_call")
agent = agent_builder.compile()

user_messages={}

INFO: Primary LLM set to Groq (openai/gpt-oss-120b).
INFO: Anthropic client initialized.


In [6]:
email = "rams@gmail.com"
if email not in user_messages:
    user_messages[email] = []

messages = user_messages[email] 
messages.append(HumanMessage(content="I want book an appointment on coming monday"))
messages = agent.invoke({"messages": messages})

--- Entering LLM Call Node ---
------------------------------------------------------
================================ System Message ================================


You are a helpful assistant that manages Google Calendar availability and bookings.
**Task** is to help users with booking a slot in the google calendar
You know to book a slot you neeed to have informations about user preference of Date & Time
You also need to have the information about whether the asked slot is available or not in the calendar.
You have the following tools at your disposal: `get_slots`, `book_slot`, and `get_current_date`.

**DO NOT MENTION THE TIMEZONE AND ASK USER**

**YOU OUTPUT WHAT YOU ARE THINKING WHEN YOU GIVE THE CONTENT**
You are very polite, professional, and always try to assist the user with the best of your ability.

================================ Human Message =================================

I want book an appointment on coming monday
------------------------------------------------

BadRequestError: Error code: 400 - {'error': {'message': "Tool call validation failed: tool call validation failed: attempted to call tool 'get_slots' which was not in request.tools", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '{"name": "get_slots", "arguments": {\n  "date": "2025-12-14"\n}}'}}